In [4]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
from ensemble_processing import load_data, load, save

LABEL_COLUMN = "Future8WeekReturn"

/Users/neilkloot/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
pd.options.display.max_rows=500

In [3]:
# df1 = pd.read_pickle('../data/ml-20180512-processed.pkl.gz', compression='gzip')
# df2 = pd.read_pickle('../data/ml-20180512-labelled.pkl.gz', compression='gzip')
# df1 =  load_data('../data/ml-20180512-processed.pkl.gz', False)

df1 = load_data('../data/ml-20180512-processed.pkl.gz',
                               drop_unlabelled=False,
                               drop_labelled=True,
                               generate_labels=False,
                               label_weeks=8,
                               reference_date='2018-05-12')

Loading file: ../data/ml-20180512-processed.pkl.gz
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3332090 entries, 2007-07-02 to 2018-05-11
Columns: 224 entries, symbol to H05_GLFSEPTPOP_T11_20P
dtypes: category(15), datetime64[ns](9), float32(129), float64(63), int32(1), int8(7)
memory usage: 3.5 GB
None
Converting dates to datetime types
Dropping columns: ['Price200DayAverage', 'Price52WeekPercChange', 'AverageVolume', 'EBITDMargin', 'EPSGrowthRate10Years', 'EPSGrowthRate5Years', 'IAD', 'LTDebtToEquityQuarter', 'LTDebtToEquityYear', 'NetIncomeGrowthRate5Years', 'NetProfitMarginPercent', 'PriceToBook', 'ReturnOnAssets5Years', 'ReturnOnAssetsTTM', 'ReturnOnAssetsYear', 'ReturnOnEquity5Years', 'ReturnOnEquityTTM', 'RevenueGrowthRate10Years', 'RevenueGrowthRate5Years', 'TotalDebtToAssetsQuarter', 'TotalDebtToAssetsYear', 'TotalDebtToEquityQuarter', 'bookValue', 'earningsPerShare', 'ebitda', 'epsEstimateCurrentYear', 'marketCapitalization', 'peRatio', 'pegRatio', 'pricePerBook', 'pr

In [ ]:
df1.columns.values

In [ ]:
# df1[['quoteDate_YEAR', 'quoteDate_MONTH', 'quoteDate_DAY']]
# pd.to_datetime(date_df)

In [23]:
df2 = pd.read_csv('../data/symbol-industry-lookup.csv')

In [24]:
df2

,symbol,GICSSector,GICSIndustryGroup,GICSIndustry
0,MOQ,Information Technology,Software & Services,Software
1,1PG,Information Technology,Software & Services,Software
2,ONT,Health Care,Health Care Equipment & Services,Health Care Providers & Services
3,14D,Unknown,Unknown,Unknown
4,1ST,Health Care,Health Care Equipment & Services,Health Care Technology
5,T3D,Industrials,Commercial & Professional Services,Commercial Services & Supplies
6,TGP,Real Estate,Real Estate,Equity Real Estate Investment Trusts (REITs)
7,TOT,Financials,Real Estate,Real Estate Investment Trusts (REITs)
8,TDO,Energy,Energy,"Oil, Gas & Consumable Fuels"
9,DDD,Materials,Materials,Metals & Mining


In [31]:
len(df2['GICSIndustryGroup'].unique())

26

In [14]:
len(df1)

49726

In [25]:
df3 = df1.merge(df2, left_on='symbol', right_on='symbol', how='left')
df3[df3['GICSSector'].isnull()]['symbol'].unique()

array([], dtype=object)

In [26]:
df3

,symbol,lastTradePriceOnly,adjustedPrice,volume,daysHigh,daysLow,previousClose,change,changeInPercent,52WeekHigh,...,H05_GLFSEPTPOP_T11_20P,exDividendRelative,quoteDate_TIMESTAMP,quoteDate_YEAR,quoteDate_MONTH,quoteDate_DAY,quoteDate_DAYOFWEEK,GICSSector,GICSIndustryGroup,GICSIndustry
0,LNY,0.003000,0.003000,1500024,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000617,NaN,353808000,2018,3,19,0,Materials,Materials,Metals & Mining
1,LNY,0.003000,0.003000,3915721,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000617,NaN,353894400,2018,3,20,1,Materials,Materials,Metals & Mining
2,LNY,0.003000,0.003000,2166677,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000617,NaN,354067200,2018,3,22,3,Materials,Materials,Metals & Mining
3,LNY,0.003000,0.003000,444279,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000688,NaN,354412800,2018,3,26,0,Materials,Materials,Metals & Mining
4,LNY,0.003000,0.003000,15722270,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000688,NaN,354499200,2018,3,27,1,Materials,Materials,Metals & Mining
5,LNY,0.003000,0.003000,1736750,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000688,NaN,354585600,2018,3,28,2,Materials,Materials,Metals & Mining
6,LNY,0.003000,0.003000,3920353,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000688,NaN,355104000,2018,4,3,1,Materials,Materials,Metals & Mining
7,LNY,0.003000,0.003000,6246402,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000379,NaN,355190400,2018,4,4,2,Materials,Materials,Metals & Mining
8,LNY,0.003000,0.003000,2270706,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000070,NaN,355363200,2018,4,6,4,Materials,Materials,Metals & Mining
9,LNY,0.003000,0.003000,1000000,0.003000,0.003000,0.003000,NaN,NaN,0.006000,...,-0.000070,NaN,355622400,2018,4,9,0,Materials,Materials,Metals & Mining


In [ ]:
from execute_xgboost_predictions import prepare_data_for_model

df_all_x, df_symbol_date = prepare_data_for_model(df1, False)

In [ ]:
df_symbol_date

In [ ]:
df_symbol_date.to_csv('test.csv')

In [58]:
categories_df = pd.read_csv('../data/symbol-industry-lookup.csv')

In [43]:
categories_df.head()

,symbol,GICSSector,GICSIndustryGroup,GICSIndustry
0,ABV,Consumer Discretionary,Automobiles & Components,Auto Components
1,ARB,Consumer Discretionary,Automobiles & Components,Auto Components
2,DDT,Consumer Discretionary,Automobiles & Components,Auto Components
3,GUD,Consumer Discretionary,Automobiles & Components,Auto Components
4,PWH,Consumer Discretionary,Automobiles & Components,Auto Components


In [7]:
train_model_names = load('../data/train_x_model_names.pkl.gz')
test_model_names = load('../data/test_x_model_names.pkl.gz')

In [8]:
train_model_names

array(['OGY', 'OGY', 'OGY', ..., 'SMR', 'SMR', 'SMR'], dtype=object)

In [15]:
train_model_names_df = pd.DataFrame({"symbol": train_model_names})

In [16]:
train_model_names_df.head()

,symbol
0,OGY
1,OGY
2,OGY
3,OGY
4,OGY


In [17]:
test_model_names_df = pd.DataFrame({"symbol": test_model_names})

In [18]:
test_model_names_df.head()

,symbol
0,OGY
1,OGY
2,OGY
3,OGY
4,OGY


In [19]:
def append_industry_categories(df: pd.DataFrame, catgories_df: pd.DataFrame):
    """
        Adds the industry categpry (GICS) values to the dataset by matching on symbol
    """

    print('Adding industry categories to data set')
    output_df = df.merge(catgories_df, left_on='symbol',
                         right_on='symbol', how='left')

    return output_df


In [59]:
train_industry_cats_df = append_industry_categories(train_model_names_df, categories_df)

Adding industry categories to data set


In [45]:
train_industry_cats_df.head(5000)

,symbol,GICSSector,GICSIndustryGroup,GICSIndustry
0,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
1,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
2,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
3,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
4,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
5,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
6,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
7,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
8,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"
9,OGY,Energy,Energy,"Oil, Gas & Consumable Fuels"


In [60]:
train_industry_cats_df.isnull().sum()

symbol                   0
GICSSector           18053
GICSIndustryGroup    18053
GICSIndustry         18053
dtype: int64

In [61]:
train_industry_cats_df[train_industry_cats_df.isnull().any(axis=1)]

,symbol,GICSSector,GICSIndustryGroup,GICSIndustry
15148,generic,NaN,NaN,NaN
15149,generic,NaN,NaN,NaN
15150,generic,NaN,NaN,NaN
15151,generic,NaN,NaN,NaN
15152,generic,NaN,NaN,NaN
15153,generic,NaN,NaN,NaN
15154,generic,NaN,NaN,NaN
15155,generic,NaN,NaN,NaN
15156,generic,NaN,NaN,NaN
15157,generic,NaN,NaN,NaN


In [64]:
train_industry_cats_df[train_industry_cats_df.isnull().any(axis=1)]['symbol'].unique()

array([], dtype=object)

In [63]:
train_industry_cats_df.fillna('Unknown', inplace=True)

In [65]:
test_industry_cats_df = append_industry_categories(test_model_names_df, categories_df)
test_industry_cats_df.fillna('Unknown', inplace=True)
test_industry_cats_df[train_industry_cats_df.isnull().any(axis=1)]['symbol'].unique()

Adding industry categories to data set


/Users/neilkloot/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


array([], dtype=object)

In [66]:
def convert_file_string(original_name):
    output_str = original_name.replace('&', 'And')
    output_str = output_str.replace(',', '')
    output_str = output_str.replace('-', '')
    output_str = output_str.replace('(', '')
    output_str = output_str.replace(')', '')
    output_str = output_str.replace(' ', '')
    return output_str


def fix_categorical(categorical_data):
    unique_vals = categorical_data.unique()
    category_renamer = {}

    for index, element in enumerate(unique_vals):
        category_renamer[element] = convert_file_string(element)

    print('Converting categories')
    print(category_renamer)

    return categorical_data.rename_categories(category_renamer)

In [72]:
train_gics_sectors_opt = pd.DataFrame()
test_gics_sectors_opt = pd.DataFrame()

train_gics_sectors_opt['symbol'] = train_industry_cats_df['symbol']
train_gics_sectors_opt['GICSSector'] = train_industry_cats_df['GICSSector'].astype('category')
train_gics_sectors_opt['GICSIndustryGroup'] = train_industry_cats_df['GICSIndustryGroup'].astype('category')
train_gics_sectors_opt['GICSIndustry'] = train_industry_cats_df['GICSIndustry'].astype('category')

test_gics_sectors_opt['symbol'] = test_industry_cats_df['symbol']
test_gics_sectors_opt['GICSSector'] = test_industry_cats_df['GICSSector'].astype('category')
test_gics_sectors_opt['GICSIndustryGroup'] = test_industry_cats_df['GICSIndustryGroup'].astype('category')
test_gics_sectors_opt['GICSIndustry'] = test_industry_cats_df['GICSIndustry'].astype('category')

train_gics_sectors = train_gics_sectors_opt['GICSSector'].values
train_gics_industry_groups = train_gics_sectors_opt['GICSIndustryGroup'].values
train_gics_industries = train_gics_sectors_opt['GICSIndustry'].values

test_gics_sectors = test_gics_sectors_opt['GICSSector'].values
test_gics_industry_groups = test_gics_sectors_opt['GICSIndustryGroup'].values
test_gics_industries = test_gics_sectors_opt['GICSIndustry'].values

# Fix the names used in the GICS data - remove '&' ',' and ' '
train_gics_sectors = fix_categorical(train_gics_sectors)
train_gics_industry_groups = fix_categorical(
    train_gics_industry_groups)
train_gics_industries = fix_categorical(train_gics_industries)

test_gics_sectors = fix_categorical(test_gics_sectors)
test_gics_industry_groups = fix_categorical(test_gics_industry_groups)
test_gics_industries = fix_categorical(test_gics_industries)

save(train_gics_sectors, '../data/train_x_GICSSector.pkl.gz')
save(train_gics_industry_groups, '../data/train_x_GICSIndustryGroup.pkl.gz')
save(train_gics_industries, '../data/train_x_GICSIndustry.pkl.gz')

save(test_gics_sectors, '../data/test_x_GICSSector.pkl.gz')
save(test_gics_industry_groups, '../data/test_x_GICSIndustryGroup.pkl.gz')
save(test_gics_industries, '../data/test_x_GICSIndustry.pkl.gz')

Converting categories
{'Energy': 'Energy', 'Unknown': 'Unknown', 'Financials': 'Financials', 'Industrials': 'Industrials', 'Materials': 'Materials', 'Information Technology': 'InformationTechnology', 'Consumer Discretionary': 'ConsumerDiscretionary', 'Real Estate': 'RealEstate', 'Health Care': 'HealthCare', 'Consumer Staples': 'ConsumerStaples', 'ETF': 'ETF', 'Telecommunication Services': 'TelecommunicationServices', 'Utilities': 'Utilities'}
Converting categories
{'Energy': 'Energy', 'Unknown': 'Unknown', 'Diversified Financials': 'DiversifiedFinancials', 'Capital Goods': 'CapitalGoods', 'Materials': 'Materials', 'Software & Services': 'SoftwareAndServices', 'Insurance': 'Insurance', 'Technology Hardware & Equipment': 'TechnologyHardwareAndEquipment', 'Retailing': 'Retailing', 'Real Estate': 'RealEstate', 'Pharmaceuticals, Biotechnology & Life Sciences': 'PharmaceuticalsBiotechnologyAndLifeSciences', 'Food, Beverage & Tobacco': 'FoodBeverageAndTobacco', 'Commercial & Professional Serv